In [19]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

In [23]:
# get song data in terms of audio features and track uri and put into data frame 
import pickle 
with open('audio_features.pickle', 'rb') as handle:
    audio_features = pickle.load(handle)
    
audio_features_df = pd.DataFrame.from_dict(audio_features, orient='index')

# get set of songs to get song names and artists 
with open('song_set.pickle', 'rb') as handle:
    song_set = pickle.load(handle)

In [239]:
print(audio_features_df.iloc[250658])
print(tracks.index('spotify:track:1lDpiiuPKTJwj7dTGlJt13'))
print(song_list[517127])

accousticness         0.9820
danceability          0.1590
energy                0.0262
instrumentalness      0.1210
liveness              0.1190
loudness            -31.5080
speechiness           0.0450
tempo               133.3320
valence               0.0332
Name: spotify:track:1lDpiiuPKTJwj7dTGlJt13, dtype: float64
517127
("L'enfance du Christ: Part III, L’Arrivée à Saïs - epilogue - O mon âme", 'spotify:track:1lDpiiuPKTJwj7dTGlJt13', 'Hector Berlioz')


In [24]:
# gettig just the track uri's 
song_list = list(song_set)
tracks = []
for song in song_list:
    tracks.append(song[1])

# get audio features into numpy array format 
audio_features_np = audio_features_df.to_numpy()

In [20]:
# creating our own knn function to compare the input song values with each song in the data frame 
def knn(data, query, k, distance_fn, choice_fn):
    neighbor_distances_and_indices = []
    
    for index, example in enumerate(data):
        distance = distance_fn(example, query)
        
        neighbor_distances_and_indices.append((distance, index))
    
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices, reverse = True)
    
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    k_nearest_labels = [data[i][0] for distance, i in k_nearest_distances_and_indices]

    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels)

# our similarity function 
def cosine_similarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

In [30]:
# query is the input vector that the similarity is being found for 
query = [0.9820, .1590, .0262, .1210, .1190, -.3155080, .0450, 133.3320, 0.0332]

# k is the number of similar songs we want to generate 
k = 5
dist_index, labels = knn(audio_features_np, query, k, cosine_similarity, lambda x: None)
print(dist_index)

[(0.9999873831978005, 250658), (0.999984327433068, 1456960), (0.9999808816574092, 906297), (0.9999806414818998, 1213216), (0.9999799380128723, 649844)]


In [32]:
# open predicted_audio_features.txt and convert to a list of lists
with open('predicted_audio_features.txt', 'r') as f:
    predicted_audio_features = f.read()

predicted_audio_features = predicted_audio_features.split('\n')
song_recs = []
for query in predicted_audio_features:
    query = [float(i) for i in eval(query)]
    dist_index, labels = knn(audio_features_np, query, 5, cosine_similarity, lambda x: None)
    # now get the track uri's of the top songs 
    target_array = audio_features_df.index.tolist()

    # get just uris 
    uris = []
    for x in dist_index:
        ind = x[1]
        uris.append(target_array[ind])


    # get the indexes of the recommendations 
    index_recs = []
    for y in uris:
        index_recs.append(tracks.index(y))


    # go back and get the name and artist 
    song_names = []
    artists = []
    for z in index_recs:
        song_names.append(song_list[z][0])
        artists.append(song_list[z][2])

    song_recs.append(song_names)

    # print out the song and artist recommendations 
    print("Here are your top " + str(k) + " recommendations!")
    for w in range(k):
        print(str(w+1) + ": " + song_names[w] + " by " + artists[w])
    print("\n")

Here are your top 5 recommendations!
1: Xmas Curtain by My Morning Jacket
2: My Cousin, All Grown Up by Our Griffins
3: Dimentica Paris by Marchio Bossa
4: New Big Prinz by The Fall
5: The Peasant by Weather Report
Here are your top 5 recommendations!
1: My Cousin, All Grown Up by Our Griffins
2: Xmas Curtain by My Morning Jacket
3: New Big Prinz by The Fall
4: The Peasant by Weather Report
5: Purple Tree by June Marieezy
Here are your top 5 recommendations!
1: My Cousin, All Grown Up by Our Griffins
2: Xmas Curtain by My Morning Jacket
3: New Big Prinz by The Fall
4: The Peasant by Weather Report
5: Purple Tree by June Marieezy
Here are your top 5 recommendations!
1: My Cousin, All Grown Up by Our Griffins
2: Xmas Curtain by My Morning Jacket
3: New Big Prinz by The Fall
4: The Peasant by Weather Report
5: Purple Tree by June Marieezy
Here are your top 5 recommendations!
1: My Cousin, All Grown Up by Our Griffins
2: Xmas Curtain by My Morning Jacket
3: New Big Prinz by The Fall
4: The

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)